In [2]:
# import
%matplotlib inline
!curl https://colab.chainer.org/install | sh -
!pip install chutil
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import chainer
import chutil

import chainer.functions as F
import chainer.links as L
from chainer import Chain
import chainer.optimizers as optimizers

from chainer.datasets.cifar import get_cifar100
from chainer import optimizers, training
from chainer.training import extensions

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1379  100  1379    0     0   8307      0 --:--:-- --:--:-- --:--:--  8307
+ apt -y -q install cuda-libraries-dev-9-2
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-9-2 is already the newest version (9.2.148-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
+ pip install -q cupy-cuda92  chainer 
+ set +ex
Installation succeeded!


In [3]:
# データセットがダウンロード済みでなければ、ダウンロードも行う
train, test = get_cifar100()
train, validation = chainer.datasets.split_dataset_random(train, 40000, seed=0)

In [ ]:
class MyConvNet(Chain):
    def __init__(self):
        super(MyConvNet, self).__init__()
        with self.init_scope():
            # 畳み込み層の定義
            # in_channels:Noneを指定しても動的にメモリ確保するので問題なく動作する
            # out_channels:出力する配列のチャンネル数
            # ksize:フィルタのサイズ（平行移動するフィルターの長さを指定）
            # stride:入力データに対してstride分フィルターを適用していくパラメータを指定
            # pad:イメージは画像データの周りにpadのサイズ分だけ空白を用意してそこに対してもフィルターを適用するようなイメージ
            # dilate:今回の実装では設定していないが、飛び飛びにフィルターを適用するパラメータ
            self.conv1 = L.Convolution2D(
                in_channels=None, out_channels=32, ksize=3, stride=1, pad=1)
            # 畳み込み層の定義２層目
            self.conv2 = L.Convolution2D(
                in_channels=None, out_channels=64, ksize=3, stride=1, pad=1)
            # 畳み込み層の定義３層目
            self.conv3 = L.Convolution2D(
                in_channels=None, out_channels=128, ksize=3, stride=1, pad=1)
            self.conv4 = L.Convolution2D(
                in_channels=None, out_channels=256, ksize=3, stride=1, pad=1)
            self.conv5 = L.Convolution2D(
                in_channels=None, out_channels=256, ksize=3, stride=1, pad=1)
            self.fc6 = L.Linear(None, 1000)
            self.fc7 = L.Linear(None, 100)

    def __call__(self, x):
        h = F.relu(self.conv1(x.reshape((-1, 3, 32, 32))))
        h = F.max_pooling_2d(h, ksize=2, stride=2)
        h = F.relu(self.conv2(h))
        h = F.max_pooling_2d(h, ksize=2, stride=2)
        h = F.relu(self.conv3(h))
        h = F.max_pooling_2d(h, ksize=2, stride=2)
        h = F.relu(self.conv4(h))
        h = F.relu(self.conv5(h))
        h = F.relu(self.fc6(h))
        return self.fc7(h)

In [ ]:
def  train_and_validate(
        model, optimizer, train, validation, n_epoch, batchsize, device=0):
    
    # 1. deviceがgpuであれば、gpuにモデルのデータを転送する
    if device >= 0:
        model.to_gpu(device)
        
    # 2. Optimizerを設定する
    optimizer.setup(model)
    
    # 3. DatasetからIteratorを作成する
    train_iter = chainer.iterators.SerialIterator(train, batchsize)
    validation_iter = chainer.iterators.SerialIterator(
        validation, batchsize, repeat=False, shuffle=False)
    
    # 4. Updater・Trainerを作成する
    updater = training.StandardUpdater(train_iter, optimizer, device=device)
    trainer = chainer.training.Trainer(updater, (n_epoch, 'epoch'), out='out')
    
    # 5. Trainerの機能を拡張する
    trainer.extend(extensions.LogReport())
    trainer.extend(extensions.Evaluator(validation_iter, model, device=device), name='val')
    trainer.extend(extensions.PrintReport(
        ['epoch', 'main/loss', 'main/accuracy', 'val/main/loss', 'val/main/accuracy', 'elapsed_time']))
    trainer.extend(extensions.PlotReport(
        ['main/loss', 'val/main/loss'],x_key='epoch', file_name='loss.png'))
    trainer.extend(extensions.PlotReport(
        ['main/accuracy', 'val/main/accuracy'], x_key='epoch', file_name='accuracy.png'))
    trainer.extend(extensions.dump_graph('main/loss'))
    
    # 6. 訓練を開始する
    trainer.run()

In [ ]:
# test 結果計算
def show_test_performance(model, test, batchsize, device=0):
    if device >= 0:
        model.to_gpu()
        
    test_iter = chainer.iterators.SerialIterator(
        test, batchsize, repeat=False, shuffle=False
    )
    test_evaluator = extensions.Evaluator(test_iter, model, device=device)
    results = test_evaluator()
    print("Test accuracy:", results["main/accuracy"])

In [7]:
n_epoch = 40
batchsize = 128

model = MyConvNet()
classifier_model = L.Classifier(model)
optimizer = optimizers.Adam()
train_and_validate(
    classifier_model, optimizer, train, validation, n_epoch, batchsize)
show_test_performance(classifier_model, test, batchsize)

epoch       main/loss   main/accuracy  val/main/loss  val/main/accuracy  elapsed_time
1           4.06144     0.0731829      3.60926        0.13835            21.8792       
2           3.24881     0.208934       3.08614        0.238331           35.0247       
3           2.77216     0.296426       2.77612        0.303402           48.1662       
4           2.4122      0.370393       2.65091        0.329213           61.2678       
5           2.08473     0.439072       2.50608        0.355914           74.377        
6           1.74294     0.51883        2.51225        0.370649           87.4692       
7           1.37447     0.608626       2.68932        0.379055           100.541       
8           1.00209     0.703726       2.88676        0.368374           113.672       
9           0.672369    0.794753       3.31775        0.377077           126.779       
10          0.427443    0.865485       3.81099        0.365012           139.878       
11          0.293868    0.906699  